# 04 CNN Maxpooling

In [ ]:
import config
from utils import data as utils_data
from utils.model import Model

import pandas as pd
import numpy as np
import tensorflow as tf

## Load data

In [ ]:
data, imgs_left, imgs_right = utils_data.load(
    config.PATH_DATA, config.PATH_IMGS_LEFT, config.PATH_IMGS_RIGHT
)

In [ ]:
data.head()

## Split data

In [ ]:
(
    (train_data, train_imgs_left, train_imgs_right),
    (validation_data, validation_imgs_left, validation_imgs_right),
    (test_data, test_imgs_left, test_imgs_right)
) = utils_data.split(
    data, imgs_left, imgs_right,
    train_size=0.95,
    validation_size=0.95,
    random_state=42
)

In [ ]:
print("Train length: {}".format(len(train_data)))
print("Validation length: {}".format(len(validation_data)))
print("Test length: {}".format(len(test_data)))

## Model

### Architecture

In [ ]:
def get_model(features, left_imgs, right_imgs, training, keep_prob=0.7):
    new_shape = np.array([-1, 20, 30, 1])
    with tf.variable_scope('model'):
        # Left Eye Img
        l_input = tf.reshape(left_imgs, new_shape)
        # 20x30x1
        cnn_l_01 = tf.layers.conv2d(
            inputs=l_input, 
            filters=32, 
            kernel_size=5, 
            strides=1,
            padding="SAME"
        )
        # 20x30x32
        pool_l_01 = tf.layers.max_pooling2d(
            inputs=cnn_l_01,
            pool_size=[2, 2],
            strides=2
        )
        # 10x15x32
        cnn_l_02 = tf.layers.conv2d(
            inputs=pool_l_01, 
            filters=128, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        )  
        # 4x7x128
        
        # Right Eye Img  
        r_input = tf.reshape(right_imgs, new_shape)      
        # 20x30x1
        cnn_r_01 = tf.layers.conv2d(
            inputs=r_input, 
            filters=32, 
            kernel_size=5, 
            strides=1,
            padding="SAME"
        )
        # 20x30x32
        pool_r_01 = tf.layers.max_pooling2d(
            inputs=cnn_r_01,
            pool_size=[2, 2],
            strides=2
        )
        # 10x15x32 
        cnn_r_02 = tf.layers.conv2d(
            inputs=pool_r_01, 
            filters=128, 
            kernel_size=3, 
            strides=2,
            padding="VALID"
        )
        # 4x7x128
        
        # Flatten convs, concat & dense        
        left_flat = tf.contrib.layers.flatten (cnn_l_02)
        right_flat =  tf.contrib.layers.flatten (cnn_r_02)
        img_concat = tf.concat(
            values=[left_flat, right_flat],
            axis=1
        )
        img_dense = tf.layers.dense(
            inputs=img_concat,
            units=128,
            activation=tf.nn.relu,
        )
        img_dropout = tf.nn.dropout(
            x=img_dense,
            keep_prob=keep_prob
        )

        
        # Concat imgs with features, dense x 2 and output
        global_concat = tf.concat(
            values=[features, img_dropout],
            axis=1
        )
        global_dense01 = tf.layers.dense(global_concat, 128, activation=tf.nn.relu)
        global_dropout01 = tf.nn.dropout(
            x=global_dense01,
            keep_prob=keep_prob
        )
        global_dense02 = tf.layers.dense(global_dropout01, 64, activation=tf.nn.relu)
        global_dropout02 = tf.nn.dropout(
            x=global_dense02,
            keep_prob=keep_prob
        )
        global_dense03 = tf.layers.dense(global_dropout01, 2, activation=None)
        
        return global_dense03

### Parameters

In [ ]:
MODEL_NAME = 'cnn_maxpooling-01'

EPOCHS = 50
BATCH_SIZE = 512
LEARNING_RATE = 0.001
KEEP_PROB = 0.85

## Runs

In [ ]:
model = Model(MODEL_NAME, get_model)

### Train

In [ ]:
model.train(
    train_data, train_imgs_left, train_imgs_right,
    validation_data, validation_imgs_left, validation_imgs_right,
    BATCH_SIZE, EPOCHS, LEARNING_RATE, KEEP_PROB
)

### Test

In [ ]:
model_test = Model(MODEL_NAME, saved_model=MODEL_NAME+".final")

In [ ]:
model_test.test(
    test_data, test_imgs_left, test_imgs_right
)

### Results


| Name | Epochs | Batch Size | Learning rate  | Keep_prob | Train | Validation | Test |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| cnn_maxpooling-01 | 25 | 256 | 0.003 | 0.9 | 0.2498006969690323 | 0.2947801351547241 | 0.28778201 |
| cnn_maxpooling-02 | 50 | 512 | 0.001 | 0.85 | 0.18449458479881287 | 0.2521626949310303 | 0.24551259 |
